# PGATour.com Web Scraper
### Updated: 1/24/2024 to handle the new website from late 2022

#### About the Site:
The PGA Tour website had recently gone through an update in its workings as I realized upon taking on this project. To complete the scraping as I had desired, common scraping libraries like BeautifulSoup and Selenium wouldn't work as far as I was aware. Because of this, this notebook follows a journey in learning about tables built on websites through JavaScript and scraping the data utlizing this fact.

The site utilizes GraphQL for their databases, so accessing the Network tab of Google Chrome gave insight into a couple key observations regarding its workings. First, each stat has a respective "stat_id", a number indicating which stat is being reported in the table. Also, corresponding to the drop down menus accessible to the user, year and tournament can be selected. In this way, the user can indicate which year, and through which tournament they want to observe the leaderboard for any given statistic. 

In my application of this data, I wanted to find the yearly leaderboard at the end of each competitive season, i.e. after the TOUR Championship. This notebook aims to create an SQL database in which for each year, each PGA pro has a row corresponding to the values of each of their statistics that given year.

#### Procedure:
1. Import relevant libraries
2. Define a method to obtain all "Stat IDs" from https://www.pgatour.com/stats
3. Define a method to obtain a dataframe of all players on tour and their specific stat given a Stat ID and Year
4. Merge dataframes into one dataframe
5. Convert merged dataframe to SQL database for use in subsequent notebooks

## 1. Import Relevant Libraries

In [36]:
# in this notebook, requests will be utilized to access the website's source code and pandas is required as a
# data manipulation tool
import requests
import pandas as pd
from numpy import NaN

import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Access the API key from the environment variable
X_API_KEY = os.getenv("X_API_KEY")

## 2. Define get_stat_ids() Method to Obtain All Stat IDs

In [37]:
# get_stat_ids() is a method designed to create and return a dictionary of all stats on the PGATour website 
# and their corresponding IDs
def get_stat_ids():
    # this graphQL payload retrieves statistical information based on the given query
    # can be found in the Network tab of Chrome after utilizing the Inspect tool
    payload = {
        "operationName": "StatOverview",
        "variables": {
            "tourCode": "R",
            "year": 2024
        },
        "query": "query StatOverview($tourCode: TourCode!, $year: Int) {\n  statOverview(tourCode: $tourCode, year: $year) {\n    tourCode\n    year\n    categories {\n      category\n      displayName\n      subCategories {\n        displayName\n        stats {\n          statId\n          statTitle\n        }\n      }\n    }\n    stats {\n      statName\n      tourAvg\n      statId\n      players {\n        statId\n        playerId\n        statTitle\n        statValue\n        playerName\n        rank\n        country\n        countryFlag\n      }\n    }\n  }\n}"
    }

    page = requests.post("https://orchestrator.pgatour.com/graphql", json=payload, headers={"x-api-key": X_API_KEY})

    data = page.json()["data"]["statOverview"]["categories"]

    dict = {}
    
    # nested for loops parse the json data to find the title and corresponding ID of each statistic
    # and inserts it into a Python dictionary
    for category in data:
        #print(category['category'])
        for subcategory in category['subCategories']:
            #print(f"\t {subcategory['displayName']}")
            for stat in subcategory['stats']:
                #print(f"\t\t{stat['statTitle']}, {stat['statId']}")
                dict[stat['statTitle']] = stat['statId']
    return(dict)

## 3. Define get_df Method to Obtain a Data Frame for a Given Statistic

In [43]:
# get_df() is a method that takes in a given stat id and year as parameters and creates and returns a data frame
# storing each pro on tour that year and their performance in that given stat
def get_df(stat_id, year):

    STAT_ID = stat_id
    YEAR = year
    # this tournament ID corresponds to the TOUR championship
    TOURNAMENT_ID = "R2023060"

    # for the 2022-23 season, the drop down menu contains tournaments after the end of the tour season
    # the if statement covers this case making the table report values only through the end of the season
    # ignoring values after for consistency
    if year == 2023:
        payload = {
            "operationName": "StatDetails",
            "variables": {
                "tourCode": "R",
                "statId": STAT_ID,
                "year": YEAR,
                "eventQuery": {
                    "queryType": "THROUGH_EVENT",
                    "tournamentId": TOURNAMENT_ID,
                }
            },
            "query": "query StatDetails($tourCode: TourCode!, $statId: String!, $year: Int, $eventQuery: StatDetailEventQuery) {\n  statDetails(\n    tourCode: $tourCode\n    statId: $statId\n    year: $year\n    eventQuery: $eventQuery\n  ) {\n    tourCode\n    year\n    displaySeason\n    statId\n    statType\n    tournamentPills {\n      tournamentId\n      displayName\n    }\n    yearPills {\n      year\n      displaySeason\n    }\n    statTitle\n    statDescription\n    tourAvg\n    lastProcessed\n    statHeaders\n    statCategories {\n      category\n      displayName\n      subCategories {\n        displayName\n        stats {\n          statId\n          statTitle\n        }\n      }\n    }\n    rows {\n      ... on StatDetailsPlayer {\n        __typename\n        playerId\n        playerName\n        country\n        countryFlag\n        rank\n        rankDiff\n        rankChangeTendency\n        stats {\n          statName\n          statValue\n          color\n        }\n      }\n      ... on StatDetailTourAvg {\n        __typename\n        displayName\n        value\n      }\n    }\n    sponsorLogo\n  }\n}"
        }
    else:
        payload = {
            "operationName": "StatDetails",
            "variables": {
                "tourCode": "R",
                "statId": STAT_ID,
                "year": YEAR,
            },
            "query": "query StatDetails($tourCode: TourCode!, $statId: String!, $year: Int, $eventQuery: StatDetailEventQuery) {\n  statDetails(\n    tourCode: $tourCode\n    statId: $statId\n    year: $year\n    eventQuery: $eventQuery\n  ) {\n    tourCode\n    year\n    displaySeason\n    statId\n    statType\n    tournamentPills {\n      tournamentId\n      displayName\n    }\n    yearPills {\n      year\n      displaySeason\n    }\n    statTitle\n    statDescription\n    tourAvg\n    lastProcessed\n    statHeaders\n    statCategories {\n      category\n      displayName\n      subCategories {\n        displayName\n        stats {\n          statId\n          statTitle\n        }\n      }\n    }\n    rows {\n      ... on StatDetailsPlayer {\n        __typename\n        playerId\n        playerName\n        country\n        countryFlag\n        rank\n        rankDiff\n        rankChangeTendency\n        stats {\n          statName\n          statValue\n          color\n        }\n      }\n      ... on StatDetailTourAvg {\n        __typename\n        displayName\n        value\n      }\n    }\n    sponsorLogo\n  }\n}"
        }

    page = requests.post("https://orchestrator.pgatour.com/graphql", json=payload, headers={"x-api-key": X_API_KEY})

    data = page.json()["data"]["statDetails"]["rows"]

    # creates a table parsing through JSON data and creating a dataframe        
    table = []

    # filter out items, thats __typename is not "StatDetailsPlayer" like
    # {"__typename": "StatDetailTourAvg", "displayName": "Tour Average", "value": "3.64"}
    data = filter(lambda item: item.get("__typename", NaN) == "StatDetailsPlayer", data)

    # format to a table that is in the webpage
    table = map(lambda item: {
        "player": item["playerName"],
        "Avg": item["stats"][0]["statValue"],
    }, data)
    
    df = pd.DataFrame(table)
    return df

## Establish Desired Statistics and Merge Yearly Data Frames Into One

In [46]:
# desired_stats list to input which stats are important to include in the data frame
desired_stats = ['SG: Total','SG: Tee-to-Green','SG: Off-the-Tee','SG: Approach the Green','SG: Around-the-Green',
                 'SG: Putting','Driving Distance','Driving Accuracy Percentage','Club Head Speed','Ball Speed','Smash Factor',
                 'Greens in Regulation Percentage','Scrambling', 'Proximity to Hole (ARG)','Overall Putting Average','Birdie or Better Conversion Percentage',
                'One-Putt Percentage','Scoring Average', 'Birdie Average','Par 3 Scoring Average',
                'Par 4 Scoring Average','Par 5 Scoring Average','Top 10 Finishes','Official Money','FedExCup Regular Season Points']
stat_ids = get_stat_ids()
df = pd.DataFrame()

# for each year between 2013 and 2023 create a data frame for every statistic that year and merge
# in the end, concatenate each year's dataframe to one large data frame to be converted into an SQL database
for year in range(2014, 2024):
    df_year = pd.DataFrame()
    
    df_stat = get_df(stat_ids['SG: Total'], year)
    df_stat.drop(columns=['Avg'], inplace=True)
    df_year = pd.concat([df_year, df_stat], ignore_index=True)
    
    for stat_name in desired_stats:
        df_stat = get_df(stat_ids[stat_name], year)

        # renames 'Avg' column to a descriptive title for the statistic
        df_stat.rename(columns={df_stat.columns[1]: stat_name}, inplace=True)

        df_year = pd.merge(df_year, df_stat, on='player')
#         print(stat_name)
#         print(df_stat.head())
    df_year.insert(0, 'Year', year)
#     print(year)
#     print(df_year.head())
    
    df = pd.concat([df, df_year], ignore_index=True)

,Year,player,SG: Total,SG: Tee-to-Green,SG: Off-the-Tee,SG: Approach the Green,SG: Around-the-Green,SG: Putting,Driving Distance,Driving Accuracy Percentage,...,Birdie or Better Conversion Percentage,One-Putt Percentage,Scoring Average,Birdie Average,Par 3 Scoring Average,Par 4 Scoring Average,Par 5 Scoring Average,Top 10 Finishes,Official Money,FedExCup Regular Season Points
0,2014,Rory McIlroy,2.266,1.992,1.367,.602,.022,.274,310.5,59.93,...,36.84,41.75,68.827,4.58,3.03,3.94,4.55,12,"$8,280,096","2,582"
1,2014,Sergio Garcia,1.984,1.818,.382,1.150,.286,.167,294.3,62.19,...,29.97,40.61,68.950,3.86,2.95,3.96,4.65,10,"$4,939,606","1,700"
2,2014,Jim Furyk,1.796,1.705,.121,1.175,.408,.091,279.3,73.18,...,26.65,39.10,69.200,3.44,3.01,3.98,4.63,11,"$5,987,395","1,851"
3,2014,Adam Scott,1.720,1.500,.725,.610,.166,.220,303.5,61.51,...,33.89,40.44,69.205,4.24,3.10,3.97,4.48,10,"$4,098,588","1,479"
4,2014,Matt Kuchar,1.573,1.117,.210,.527,.380,.457,283.8,67.37,...,30.57,39.96,69.416,3.82,3.04,3.97,4.65,11,"$4,695,515","1,921"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1522,2023,Harry Higgs,-1.157,-1.461,-0.521,-0.937,-0.004,0.304,302.8,52.37%,...,32.95%,42.81%,71.732,3.87,3.11,4.03,4.59,1,"$684,386",274
1523,2023,Martin Trainer,-1.366,-1.173,-0.415,-0.622,-0.137,-0.194,303.7,54.49%,...,31.80%,39.36%,72.370,3.66,3.16,4.05,4.65,1,"$333,888",139
1524,2023,Brent Grant,-1.464,-0.395,0.797,-0.680,-0.512,-1.069,310.6,62.79%,...,30.30%,34.76%,72.196,3.65,3.18,4.07,4.63,1,"$374,126",158
1525,2023,Max McGreevy,-1.695,-1.207,-0.358,-0.905,0.055,-0.488,289.8,59.51%,...,29.24%,39.91%,72.591,3.29,3.10,4.12,4.68,1,"$360,757",142


In [51]:
import sqlite3
conn = sqlite3.connect('PGATourData_13_23.db')

df.to_sql('13_23_Data', conn, if_exists='replace', index=False)

conn.close()

## Conclusion

This notebook tracks my progress--what I would consider to be leaps and bounds--in my knowledge of the workings of websites utilizing JavaScript and GraphQL to generate tables. Not only did this project allow me to reach out of my comfort zone in learning a new webscraping technique, but it gave me great practice with merging and concatenating various data frames utilizing the Pandas library. 

Overall, there is much more to be done with this notebook. For starters, one limitation that I have noticed was from 2004-2012, the aforementioned drop down doesn't have the TOUR championship as the last championship, so another condition might need to be checked. Luckly, these years are outside the scope of my data project in which I'll be using this data. 

With these limitations in mind, this notebook perfectly sets up further notebooks in this repository where I analyze this data further determining interesting patterns and trends in the world of professional golf with which I hope to improve my own amateur game.